In [18]:
import os
import re
import json
import torch
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as T
import numpy as np
from tqdm import tqdm 

In [19]:
DATASET_DIR = "../TestDataSet"
LABELS_JSON_PATH = os.path.join(DATASET_DIR, "labels_list.json")
PRETRAINED_WEIGHTS = torchvision.models.ResNet34_Weights.IMAGENET1K_V1
BATCH_SIZE = 32
NUM_WORKERS = 0

In [20]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cpu


### Model Loading and Preparation

Here we load a pre-trained ResNet-34 model and prepares it for evaluation:

- **`torchvision.models.resnet34(weights=PRETRAINED_WEIGHTS)`**:  
  Instantiates the ResNet-34 model with the specified pre-trained weights.

- **`model.eval()`**:  
  Sets the model to evaluation mode.  
  This is crucial for disabling layers like dropout and ensuring consistent behavior of batch normalization layers during inference.

In [21]:
print(f"Loading ResNet-34 model with {PRETRAINED_WEIGHTS} weights...")
model = torchvision.models.resnet34(weights=PRETRAINED_WEIGHTS)
model = model.to(DEVICE)
model.eval()
print("Model loaded and set to evaluation mode.")

Loading ResNet-34 model with ResNet34_Weights.IMAGENET1K_V1 weights...
Model loaded and set to evaluation mode.


### Image Transformations

Here we define the image transformations that will be applied to the images before feeding them to the model:

- **`MEAN_NORMS` and `STD_NORMS`**:  
  These are the mean and standard deviation values used for normalizing the image data.  
  These specific values are commonly used when working with models pre-trained on ImageNet.

- **`T.Compose([...])`**:  
  Combines multiple transformations into a single sequence.

    - **`T.ToTensor()`**:  
      Converts the image from a PIL Image or NumPy array to a PyTorch tensor and scales the pixel values to the range `[0, 1]`.

    - **`T.Normalize(mean=MEAN_NORMS, std=STD_NORMS)`**:  
      Normalizes the tensor by subtracting the mean and dividing by the standard deviation for each color channel.  
      This is a standard preprocessing step for neural networks.


In [22]:
MEAN_NORMS = np.array([0.485, 0.456, 0.406])
STD_NORMS = np.array([0.229, 0.224, 0.225])

eval_transforms = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=MEAN_NORMS, std=STD_NORMS)
])

### Dataset Loading and DataLoader Creation

This cell loads the image dataset using `ImageFolder` and prepares it with a `DataLoader`.  
Images are organized into subfolders, and the defined transformations are applied during loading.  
The `DataLoader` is configured with the specified batch size, number of workers, and uses pinned memory for efficient GPU data transfer.  
The dataset directory is also validated, and dataset information is printed.


In [23]:
# === Load Dataset using ImageFolder ===
print(f"[INFO] Loading dataset from: {DATASET_DIR}")
if not os.path.isdir(DATASET_DIR):
    raise FileNotFoundError(f"[ERROR] Dataset directory not found at '{DATASET_DIR}'.")

imagefolder_dataset = torchvision.datasets.ImageFolder(
    root=DATASET_DIR,
    transform=eval_transforms
)

test_loader = DataLoader(
    imagefolder_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

num_classes = len(imagefolder_dataset.classes)
print(f"[INFO] Dataset loaded: {len(imagefolder_dataset)} images across {num_classes} classes.")
print(f"[INFO] Class index order (alphabetical): {imagefolder_dataset.classes}")


[INFO] Loading dataset from: ../TestDataSet
[INFO] Dataset loaded: 500 images across 100 classes.
[INFO] Class index order (alphabetical): ['n02672831', 'n02676566', 'n02687172', 'n02690373', 'n02692877', 'n02699494', 'n02701002', 'n02704792', 'n02708093', 'n02727426', 'n02730930', 'n02747177', 'n02749479', 'n02769748', 'n02776631', 'n02777292', 'n02782093', 'n02783161', 'n02786058', 'n02787622', 'n02788148', 'n02790996', 'n02791124', 'n02791270', 'n02793495', 'n02794156', 'n02795169', 'n02797295', 'n02799071', 'n02802426', 'n02804414', 'n02804610', 'n02807133', 'n02808304', 'n02808440', 'n02814533', 'n02814860', 'n02815834', 'n02817516', 'n02823428', 'n02823750', 'n02825657', 'n02834397', 'n02835271', 'n02837789', 'n02840245', 'n02841315', 'n02843684', 'n02859443', 'n02860847', 'n02865351', 'n02869837', 'n02870880', 'n02871525', 'n02877765', 'n02879718', 'n02883205', 'n02892201', 'n02892767', 'n02894605', 'n02895154', 'n02906734', 'n02909870', 'n02910353', 'n02916936', 'n02917067', 'n

### Label Mapping from JSON

Now load a JSON file to map `ImageFolder` class indices to ImageNet class indices.  
It validates the loaded data, checks for mismatches in class counts, extracts indices using regular expressions, and handles errors if label parsing fails.



In [24]:
# === Load Dataset-Specific Label Mappings from JSON ===
print(f"[INFO] Loading dataset-specific labels from: {LABELS_JSON_PATH}")
imagefolder_to_imagenet_map = {}

with open(LABELS_JSON_PATH, "r") as f:
    json_labels = json.load(f)

if not isinstance(json_labels, list):
    raise ValueError("Expected a list of label strings in the JSON file.")

num_json_labels = len(json_labels)

if num_json_labels != num_classes:
    print("\n[WARNING] Class count mismatch!")
    print(f"- ImageFolder found {num_classes} classes in '{DATASET_DIR}'")
    print(f"- JSON file contains {num_json_labels} entries")
    print("[WARNING] Proceeding, but label mapping may be incorrect.")

print("[INFO] Building mapping from ImageFolder index to ImageNet index...")
parse_errors = 0

for idx in range(min(num_classes, num_json_labels)):
    label_entry = json_labels[idx]
    match = re.match(r"\s*(\d+)\s*:", label_entry)

    if match:
        imagenet_idx = int(match.group(1))
        imagefolder_to_imagenet_map[idx] = imagenet_idx
    else:
        print(f"[ERROR] Could not parse index from: '{label_entry}'")
        parse_errors += 1

if parse_errors > 0:
    print(f"[WARNING] {parse_errors} label parsing errors encountered. Mapping may be incomplete.")
elif not imagefolder_to_imagenet_map:
    print("[ERROR] No valid label mappings were created.")
else:
    print("[INFO] Label mapping built successfully.")


[INFO] Loading dataset-specific labels from: ../TestDataSet/labels_list.json
[INFO] Building mapping from ImageFolder index to ImageNet index...
[INFO] Label mapping built successfully.


### Evaluation Loop

We perform the evaluation of the model:

- Using `torch.no_grad()` to disable gradient calculation for efficient inference.
- Iterate over batches from `test_loader` with a progress bar using `tqdm`.
- Move each batch of images to the selected device (`cuda` or `cpu`).
- Map `ImageFolder` labels to ImageNet indices, filtering out any samples without a valid mapping.
- Feed valid images to the model to obtain output logits.
- Extract Top-5 predictions for each image using `torch.topk`.
- Calculate Top-1 accuracy by checking if the top prediction matches the target label.
- Calculate Top-5 accuracy by checking if the target label is within the top 5 predictions.
- Update counters for each batch and, after all batches, computes and prints the overall Top-1 and Top-5 accuracies.


In [25]:

top1_correct = 0
top5_correct = 0
total_samples = 0

print("[INFO] Starting evaluation...")
with torch.no_grad():
    for images, folder_labels in tqdm(test_loader, desc="Evaluating Batches"):
        images = images.to(DEVICE)

        # Map ImageFolder indices to actual ImageNet indices
        valid_indices = []
        true_imagenet_labels = []

        for i, folder_idx in enumerate(folder_labels.tolist()):
            if folder_idx in imagefolder_to_imagenet_map:
                valid_indices.append(i)
                true_imagenet_labels.append(imagefolder_to_imagenet_map[folder_idx])

        if not valid_indices:
            continue

        images_valid = images[valid_indices]
        targets = torch.tensor(true_imagenet_labels, dtype=torch.long).to(DEVICE)

        outputs = model(images_valid)
        _, top5_preds = torch.topk(outputs, k=5, dim=1)

        total_samples += targets.size(0)

        # Top-1 Accuracy
        top1_correct += (top5_preds[:, 0] == targets).sum().item()

        # Top-5 Accuracy
        matches_top5 = (top5_preds == targets.view(-1, 1)).sum(dim=1)
        top5_correct += matches_top5.sum().item()

# Final Accuracy Results
top1_acc = top1_correct / total_samples if total_samples > 0 else 0.0
top5_acc = top5_correct / total_samples if total_samples > 0 else 0.0


[INFO] Starting evaluation...


Evaluating Batches: 100%|██████████| 16/16 [00:15<00:00,  1.02it/s]


In [9]:
# === Final Accuracy Report ===
if total_samples == 0:
    print("\nNo samples were evaluated. Check JSON structure, parsing logic, or folder setup.")
    top1_acc = 0.0
    top5_acc = 0.0
else:
    top1_acc = (top1_correct / total_samples) * 100
    top5_acc = (top5_correct / total_samples) * 100

print("\n=== Evaluation Summary ===")
print(f"Total Images Evaluated: {total_samples}")
print(f"Top-1 Accuracy: {top1_acc:.2f}% ({top1_correct}/{total_samples})")
print(f"Top-5 Accuracy: {top5_acc:.2f}% ({top5_correct}/{total_samples})")



=== Evaluation Summary ===
Total Images Evaluated: 500
Top-1 Accuracy: 76.00% (380/500)
Top-5 Accuracy: 94.00% (470/500)


### Evaluation Results Calculation

- **Top-1 Accuracy: 76.00% (380/500)**  
  Here we measure the percentage of images where the model's top prediction matches the true class.  
  - Correct predictions: 380  
  - Calculation: `(380 / 500) * 100 = 76.00%`  
  This means the model correctly identified the primary class for 76% of the images.

- **Top-5 Accuracy: 94.20% (470/500)**  
  Measure the percentage of images where the true class is among the model's top 5 predictions.  
  - Correct predictions within top 5: 470  
  - Calculation: `(470 / 500) * 100 = 94.00%`  
  This shows the model included the true class in its top 5 guesses for 94.2% of the images.

#### Summary
The results indicate that the pre-trained ResNet-34 model performs well on the test dataset, achieving 76% Top-1 and over 94% Top-5 accuracy.  
This suggests the dataset likely contains classes somewhat similar to those the model was trained on (ImageNet), indicating that the model performs well at classifying images from these 100 classes, successfully placing the correct label in its top 5 predictions for the vast majority of samples.
